# Graph charges with NAGL

NAGL is a graph neural network toolkit designed for constructing, using, and distributing models that can compute partial charges from a molecular graph. NAGL charges are computed directly from the connectivity graph, are independent of conformation, are much faster than QC methods, and scale better to larger molecules. Currently released NAGL models are trained to reproduce AM1BCC charges for molecules that Sage is designed to support.

This notebook is YELLOW because:
- NAGL is pre-release and experimental

Linezolid is a reserve antibiotic for MDR Gram-positive infections

In [ ]:
from openff.toolkit import Molecule

linezolid = Molecule.from_smiles("O=C1O[C@@H](CNC(=O)C)CN1c3cc(F)c(N2CCOCC2)cc3")

linezolid.visualize(show_all_hydrogens=False)

In [ ]:
from openff.toolkit.utils.toolkits import ToolkitRegistry, toolkit_registry_manager, RDKitToolkitWrapper, BuiltInToolkitWrapper
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

In [ ]:
NAGLToolkitWrapper().supported_charge_methods

In [ ]:
# Do a "sacrificial" first run of nagl on methane so the model is compiled in this cell
Molecule.from_smiles("C").assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.2.pt", toolkit_registry=NAGLToolkitWrapper()
)

## Compute NAGL and Antechamber charges

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

with toolkit_registry_manager(ToolkitRegistry([NAGLToolkitWrapper, RDKitToolkitWrapper, BuiltInToolkitWrapper])):
    linezolid_nagl = Molecule(linezolid)
    linezolid_nagl.assign_partial_charges('openff-gnn-am1bcc-0.1.0-rc.1.pt')

In [ ]:
linezolid_nagl.partial_charges

In [ ]:
from openff.toolkit.utils.toolkits import AmberToolsToolkitWrapper

with toolkit_registry_manager(ToolkitRegistry([ AmberToolsToolkitWrapper, RDKitToolkitWrapper, BuiltInToolkitWrapper])):
    linezolid_antechamber = Molecule(linezolid)
    linezolid_antechamber.assign_partial_charges('AM1BCC')

In [ ]:
linezolid_antechamber.partial_charges

## Compare NAGL and Antechamber charges

In [ ]:
from matplotlib import pyplot

pyplot.scatter(linezolid_nagl.partial_charges.m, linezolid_antechamber.partial_charges.m)
pyplot.ylabel(f"reference charges ({linezolid_antechamber.partial_charges.u})")
pyplot.xlabel(f"nagl charges ({linezolid_nagl.partial_charges.u})")
pyplot.axline((0,0), slope=1, linestyle=":")

In [ ]:
charge_diffs = linezolid_nagl.partial_charges - linezolid_antechamber.partial_charges
pyplot.bar(range(len(charge_diffs)), charge_diffs.m)
pyplot.ylabel(f"charge difference ({charge_diffs.u})")
pyplot.xlabel("atom index")

In [ ]:
from utils import draw_molecule

draw_molecule(linezolid, atom_notes={i: f'{i}' for i in range(linezolid.n_atoms)})